In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
import logging
import datetime
from datetime import datetime
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
# Tokenizer and stemmer initialized once
stemmer = PorterStemmer()
import concurrent.futures
from textblob import TextBlob
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import tensorflow as tf
import contractions, re
import string
from transformers import TFBertForSequenceClassification, BertTokenizer
import tensorflow as tf
from nltk.tokenize import sent_tokenize
import spacy
# Loading Spacy small model as nlp
nlp = spacy.load('en_core_web_sm')
from collections import Counter
from wordcloud import WordCloud
import math
# import warnings
# warnings.filterwarnings("ignore")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
transcripts_data = pd.read_pickle("motley-fool-data.pkl")

In [3]:
transcripts_data.head()

,date,exchange,q,ticker,transcript
0,"Aug 27, 2020, 9:00 p.m. ET",NASDAQ: BILI,2020-Q2,BILI,"Prepared Remarks:\nOperator\nGood day, and wel..."
1,"Jul 30, 2020, 4:30 p.m. ET",NYSE: GFF,2020-Q3,GFF,Prepared Remarks:\nOperator\nThank you for sta...
2,"Oct 23, 2019, 5:00 p.m. ET",NASDAQ: LRCX,2020-Q1,LRCX,Prepared Remarks:\nOperator\nGood day and welc...
3,"Nov 6, 2019, 12:00 p.m. ET",NASDAQ: BBSI,2019-Q3,BBSI,"Prepared Remarks:\nOperator\nGood day, everyon..."
4,"Aug 7, 2019, 8:30 a.m. ET",NASDAQ: CSTE,2019-Q2,CSTE,Prepared Remarks:\nOperator\nGreetings and wel...


In [4]:
transcripts_data=transcripts_data[~transcripts_data.duplicated(["transcript"])].reset_index(drop=True)

In [5]:
transcripts_data["transcript"].iloc[0]

"Prepared Remarks:\nOperator\nGood day, and welcome to the Bilibili 2020 Second Quarter Earnings Conference Call. Today's conference is being recorded.\nAt this time, I would like to turn the conference over to Juliet Yang, Senior Director of Investor Relations. Please go ahead.\nJuliet Yang -- Senior Director of Investor Relations\nThank you, operator.\nPlease note the discussion today will contain forward-looking statements relating to the Company's future performance, and are intended to qualify for the Safe Harbor from liability, as established by the US Private Securities Litigation Reform Act. Such statements are not guarantees of future performance and are subject to certain risks and uncertainties, assumptions and other factors. Some of these risks are beyond the Company's control and could cause actual results to differ materially from those mentioned in today's press release and this discussion. A general discussion of the risk factors that could affect Bilibili's business an

In [6]:
transcripts_data_google = transcripts_data[transcripts_data["ticker"]=="GOOGL"]

In [7]:
# Function to extract participants from the transcript and exlude the ones asking questions
def extract_participants_and_exclude(transcript):
    # Regular expression to capture participants under "Call participants" which we see in the end of the transcript
    participants_pattern = r"Call participants:(.*?)(?=\nMore|\nAll|\Z)"
    
    # Find participants using the pattern
    match = re.search(participants_pattern, transcript, re.DOTALL)
    if match:
        participants_text = match.group(1)
        
        # Split participants into a list
        participants_list = participants_text.strip().split("\n")
        
        # if participants with company names then they are not from the company
        included_participants = [participant for participant in participants_list if len(participant.split(" -- ")) == 2]
        excluded_participants = [participant for participant in participants_list if len(participant.split(" -- ")) != 2]
        
        return included_participants, excluded_participants
    else:
        return [], []

# Apply the function to the 'transcript' column
transcripts_data_google[['participants', 'excluded_participants']] = transcripts_data_google['transcript'].apply(lambda x: pd.Series(extract_participants_and_exclude(x)))


/var/folders/5h/1cc6fvqn1nsf3cqhcndrlz280000gn/T/ipykernel_69591/3932416784.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transcripts_data_google[['participants', 'excluded_participants']] = transcripts_data_google['transcript'].apply(lambda x: pd.Series(extract_participants_and_exclude(x)))
/var/folders/5h/1cc6fvqn1nsf3cqhcndrlz280000gn/T/ipykernel_69591/3932416784.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transcripts_data_google[['participants', 'excluded_participants']] = transcripts_da

In [8]:
transcripts_data_google

,date,exchange,q,ticker,transcript,participants,excluded_participants
313,"Jul 27, 2021, 4:30 p.m. ET",NASDAQ: GOOGL,2021-Q2,GOOGL,"Prepared Remarks:\nOperator\nWelcome, everyone...",[Jim Friedland -- Director of Investor Relatio...,"[Brian Nowak -- Morgan Stanley -- Analyst, Dou..."
634,"April 28, 2020, 4:30 p.m. ET",NASDAQ: GOOGL,2020-Q1,GOOGL,Prepared Remarks:\nOperator\nLadies and gentle...,[Jim Friedland -- Director of Investor Relatio...,"[Eric Sheridan -- UBS -- Analyst, Doug Anmuth ..."
770,"Apr 27, 2021, 5:00 p.m. ET",NASDAQ: GOOGL,2021-Q1,GOOGL,"Prepared Remarks:\nOperator\nWelcome everyone,...",[Jim Friedland -- Director of Investor Relatio...,"[Brian Nowak -- Morgan Stanley -- Analyst, Dou..."
894,"April 29, 2019, 4:30 p.m. ET",NASDAQ: GOOGL,2019-Q1,GOOGL,"Prepared Remarks:\nOperator\nGood day, ladies ...","[Ellen West -- Vice President, Investor Relati...","[Eric Sheridan -- UBS -- Analyst, Doug Anmuth ..."
954,"Feb 2, 2021, 5:00 p.m. ET",NASDAQ: GOOGL,2020-Q4,GOOGL,Prepared Remarks:\nOperator\nWelcome everyone ...,[Jim Friedland -- Director of Investor Relatio...,"[Eric Sheridan -- UBS -- Analyst, Doug Anmuth ..."
1132,[Alphabet Inc. (GOOG 0.81%) (GOOGL 0.73%)Q4 20...,NASDAQ: GOOGL,2017-Q4,GOOGL,"Prepared Remarks:\nOperator\nGood day, ladies ...","[Ellen West -- Head of Investor Relations, Rut...",[Eric J. Sheridan -- UBS Securities LLC -- Ana...
1446,"Feb 01, 2022, 5:00 p.m. ET",NASDAQ: GOOGL,2021-Q4,GOOGL,"Prepared Remarks:\nOperator\nWelcome, everyone...",[Jim Friedland -- Director of Investor Relatio...,"[Doug Anmuth -- J.P. Morgan -- Analyst, Eric S..."
1491,"Oct 26, 2021, 4:30 p.m. ET",NASDAQ: GOOGL,2021-Q3,GOOGL,"Prepared Remarks:\nOperator\nWelcome, everyone...",[James Friedland -- Director of Investor Relat...,"[Eric Sheridan -- Goldman Sachs -- Analyst, Br..."
2760,[Alphabet Inc Class C (GOOGL 0.75%) (GOOG 0.84...,NASDAQ: GOOGL,2018-Q4,GOOGL,"Prepared Remarks:\nOperator\nGood day, ladies ...","[Ellen West -- Head of Investor Relations, Rut...",[Eric Sheridan -- UBS Securities LLC -- Analys...
3281,"Jul 26, 2022, 5:00 p.m. ET",NASDAQ: GOOGL,2022-Q2,GOOGL,"Prepared Remarks:\nOperator\nWelcome, everyone...",[Jim Friedland -- Director of Investor Relatio...,"[Douglas Anmuth -- J.P. Morgan -- Analyst, Bri..."


In [9]:
transcripts_data_google["excluded_participants"].iloc[0]

['Brian Nowak -- Morgan Stanley -- Analyst',
 'Doug Anmuth -- J.P. Morgan -- Analyst',
 'Justin Post -- Bank of America -- Analyst',
 'Colin Sebastian -- Robert W. Baird -- Analyst',
 'Brent Thill -- Jefferies -- Analyst',
 'Mark Mahaney -- Evercore ISI -- Analyst',
 'Michael Nathanson -- MoffettNathanson -- Analyst',
 'Jason Bazinet -- Citi -- Analyst']

In [10]:
transcripts_data_google["participants"].iloc[0]

['Jim Friedland -- Director of Investor Relations',
 'Sundar Pichai -- Chief Executive Officer',
 'Philipp Schindler -- Chief Business Officer',
 'Ruth Porat -- Chief Financial Officer']

In [11]:
transcripts_data_google["transcript"].iloc[0]

"Prepared Remarks:\nOperator\nWelcome, everyone. Thank you for standing by for the Alphabet Second Quarter 2021 Earnings Conference Call. [Operator Instructions]\nI would now like to hand the conference over to your speaker today, Jim Friedland, Director of Investor Relations. Please go ahead.\nJim Friedland -- Director of Investor Relations\nThank you. Good afternoon everyone and welcome to Alphabet's second quarter earnings conference call. With us today are Sundar Pichai, Philipp Schindler and Ruth Porat.\nNow, I'll quickly cover the Safe Harbor. Some of the statements that we make today regarding our business, operations and financial performance, including the effect of the COVID-19 pandemic on those areas may be considered forward-looking and such statements involve a number of risks and uncertainties that could cause actual results to differ materially. For more information, please refer to the risk factors discussed in our most recent Form 10-K filed with the SEC.\nDuring this 

In [12]:
# Function to extract relevant speech, handling multiple occurrences and considering delimiters
def extract_speech(transcript, participants, excluded_participants):
    extracted_speech = {}

    # Define a regex to match any participant's speech until another participant's name or excluded names
    for participant in participants:
        # Build a dynamic regex pattern
        delimiters = participants + excluded_participants + ["Questions and Answers", "Operator"]
        delimiter_pattern = '|'.join([re.escape(d) for d in delimiters])
        pattern = re.compile(rf"({re.escape(participant)})\s*(.*?)(?=\n(?:{delimiter_pattern}|$))", re.DOTALL)
        
        # Find all matches for this participant's speech
        matches = pattern.findall(transcript)
        
        # If matches are found, clean the speech by removing other participants' names
        if matches:
            participant_speech = [match[1].strip() for match in matches]
            for other_participant in participants + excluded_participants:
                if other_participant != participant:
                    participant_speech = [re.sub(rf"\b{re.escape(other_participant)}\b", "", speech) for speech in participant_speech]
            extracted_speech[participant] = participant_speech
        else:
            extracted_speech[participant] = ["No speech found."]
    
    return extracted_speech

# Function to apply extract_speech to each row
def extract_speech_from_df(df):
    df['extracted_speech'] = df.apply(lambda row: extract_speech(row['transcript'], row['participants'], row['excluded_participants']), axis=1)
    return df


# Apply the function to the DataFrame
transcripts_data_google = extract_speech_from_df(transcripts_data_google)


/var/folders/5h/1cc6fvqn1nsf3cqhcndrlz280000gn/T/ipykernel_69591/3870756276.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['extracted_speech'] = df.apply(lambda row: extract_speech(row['transcript'], row['participants'], row['excluded_participants']), axis=1)


In [13]:
transcripts_data_google['extracted_speech_length'] = transcripts_data_google['extracted_speech'].apply(lambda speech_dict: sum(len(speech.replace("\n", " ")) for speeches in speech_dict.values() for speech in speeches))



/var/folders/5h/1cc6fvqn1nsf3cqhcndrlz280000gn/T/ipykernel_69591/119950247.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transcripts_data_google['extracted_speech_length'] = transcripts_data_google['extracted_speech'].apply(lambda speech_dict: sum(len(speech.replace("\n", " ")) for speeches in speech_dict.values() for speech in speeches))


In [14]:
len(transcripts_data_google["transcript"].iloc[0])

48912

In [15]:
transcripts_data_google["extracted_speech"].iloc[0]

{'Jim Friedland -- Director of Investor Relations': ["Thank you. Good afternoon everyone and welcome to Alphabet's second quarter earnings conference call. With us today are Sundar Pichai, Philipp Schindler and Ruth Porat.\nNow, I'll quickly cover the Safe Harbor. Some of the statements that we make today regarding our business, operations and financial performance, including the effect of the COVID-19 pandemic on those areas may be considered forward-looking and such statements involve a number of risks and uncertainties that could cause actual results to differ materially. For more information, please refer to the risk factors discussed in our most recent Form 10-K filed with the SEC.\nDuring this call, we will present both GAAP and non-GAAP financial measures. A reconciliation of non-GAAP to GAAP measures is included in today's earnings press release, which is distributed and available to the public through our Investor Relations website located at abc.xyz/investor.\nAnd now I'll tu

In [16]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

# Load FinBERT
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone', num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer)

Device set to use mps:0


In [20]:
# Function to split text into paragraphs based on single newline character
def split_text_to_paragraphs(text):
    return text.split("\n")  # Split based on single newlines

# Function to analyze sentiment for extracted speech
def analyze_extracted_speech(extracted_speech):
    participant_sentiments = {}
    
    for person, texts in extracted_speech.items():
        # Initialize probabilities
        probabilities = {"negative": 0, "neutral": 0, "positive": 0}
        
        for text in texts:
            if text.strip():  # Process non-empty text
                # print(f"Processing text: {text}")
                # print(f"Text length: {len(text)}")
                
                # Tokenize the text to check if it exceeds the token limit
                inputs = tokenizer(text, return_tensors="pt")
                
                # Check if the text exceeds the token limit
                if inputs['input_ids'].size(1) > 512:
                    # Split the text into paragraphs if it's too long
                    paragraphs = split_text_to_paragraphs(text)
                    
                    # Loop through paragraphs and analyze sentiment
                    for paragraph in paragraphs:
                        if paragraph.strip():  # Skip empty paragraphs
                            # print(f"Analyzing paragraph: {paragraph}")
                            results = nlp(paragraph)  # Assuming `nlp` is the sentiment analysis model
                            # print(results)
                            for res in results:
                                if res["label"] == "Neutral":  # Neutral
                                    probabilities["neutral"] += res["score"]
                                elif res["label"] == "Positive":  # Positive
                                    probabilities["positive"] += res["score"]
                                elif res["label"] == "Negative":  # Negative
                                    probabilities["negative"] += res["score"]
                else:
                    # Proceed with sentiment analysis for the full text
                    results = nlp(text)  # Assuming `nlp` is the sentiment analysis model
                    # print(results)
                    for res in results:
                        if res["label"] == "Neutral":  # Neutral
                            probabilities["neutral"] += res["score"]
                        elif res["label"] == "Positive":  # Positive
                            probabilities["positive"] += res["score"]
                        elif res["label"] == "Negative":  # Negative
                            probabilities["negative"] += res["score"]
                        # print("probabilities:",probabilities)
        
        # Normalize the probabilities
        total = sum(probabilities.values())
        if total > 0:
            probabilities = {k: v / total for k, v in probabilities.items()}
        
        # Find max sentiment
        max_sentiment = max(probabilities, key=probabilities.get)
        
        # Store results for the person
        participant_sentiments[person] = {
            **probabilities,
            "max_sentiment": max_sentiment
        }
    
    return participant_sentiments

transcripts_data_google["extracted_speech_sentiment"] = transcripts_data_google["extracted_speech"].apply(analyze_extracted_speech)

/var/folders/5h/1cc6fvqn1nsf3cqhcndrlz280000gn/T/ipykernel_69591/472484847.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transcripts_data_google["extracted_speech_sentiment"] = transcripts_data_google["extracted_speech"].apply(analyze_extracted_speech)


In [22]:
transcripts_data_google["extracted_speech_sentiment"].iloc[0]

{'Jim Friedland -- Director of Investor Relations': {'negative': 0.0,
  'neutral': 0.5646152370561309,
  'positive': 0.4353847629438691,
  'max_sentiment': 'neutral'},
 'Sundar Pichai -- Chief Executive Officer': {'negative': 0.0,
  'neutral': 0.35207972721928477,
  'positive': 0.6479202727807152,
  'max_sentiment': 'positive'},
 'Philipp Schindler -- Chief Business Officer': {'negative': 0.04345331118504299,
  'neutral': 0.1553802530340193,
  'positive': 0.8011664357809377,
  'max_sentiment': 'positive'},
 'Ruth Porat -- Chief Financial Officer': {'negative': 0.13544291559865212,
  'neutral': 0.1876133734710394,
  'positive': 0.6769437109303085,
  'max_sentiment': 'positive'}}

In [19]:
transcripts_data_google["extracted_speech_sentiment"].iloc[3]

{'Ellen West -- Vice President, Investor Relations': {'negative': 0.0,
  'neutral': 0.5860441752785112,
  'positive': 0.4139558247214888,
  'max_sentiment': 'neutral'},
 'Ruth Porat -- Senior Vice President and Chief Financial Officer': {'negative': 0.07540706467024873,
  'neutral': 0.4655106896740092,
  'positive': 0.45908224565574207,
  'max_sentiment': 'neutral'},
 'Sundar Pichai -- Chief Executive Officer': {'negative': 0.0,
  'neutral': 0.20441481228564037,
  'positive': 0.7955851877143596,
  'max_sentiment': 'positive'}}